<a href="https://colab.research.google.com/github/Rutuja8698/RUTUJA/blob/main/Lab6-Cats-v-Dogs.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

## Training with a Larger Dataset - Cats and Dogs

In the previous lab you trained a classifier with a horses-v-humans dataset. You saw that despite getting great training results, when you tried to do classification with real images, there were many errors, due primarily to overfitting -- where the network  does very well with data that it has previously seen, but poorly with data it hasn't!

In this lab you'll look at a real, and very large dataset, and see the impact this has to avoid overfitting.

In [4]:
import os
import zipfile
import random
import tensorflow as tf
from tensorflow.keras.optimizers import RMSprop
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from shutil import copyfile

In [8]:
import os
import zipfile
import random
import tensorflow as tf
from tensorflow.keras.optimizers import RMSprop
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from shutil import copyfile

# **Verify the Download URL:** Double-check this URL. It might be outdated.
# If the URL doesn't work, visit https://www.microsoft.com/en-us/download/confirmation.aspx?id=54765
# And right click on the 'Download Manually' link to get a new URL to the dataset
# Note: This is a very large dataset and will take time to download
!wget --no-check-certificate "https://download.microsoft.com/download/3/E/1/3E1C3F21-ECDB-4869-8368-6DEBA77B919F/kagglecatsanddogs_3367a.zip" -O "/tmp/cats-and-dogs.zip"
local_zip = '/tmp/cats-and-dogs.zip'

# Check file size to see if it downloaded completely
file_size = os.path.getsize(local_zip)
print(f"Downloaded file size: {file_size} bytes")

# If file size is 0, it means the download failed. Try troubleshooting your network connection
# or find an updated download URL.
if file_size == 0:
    # **Instead of raising an error, print a message and provide instructions.**
    print("Downloaded file is empty. Please check the download URL and your network connection.")
    print("You can find an updated download URL at: https://www.microsoft.com/en-us/download/confirmation.aspx?id=54765")
    print("Right-click on the 'Download Manually' link to get a new URL.")
    # **You can optionally add a line to halt execution here:**
    # import sys; sys.exit(1)
else:
    # Check if the file is actually a zip file
    # by checking the magic number at the beginning of the file
    with open(local_zip, 'rb') as f:
        header = f.read(4)
        if header == b'PK\x03\x04':
            print("File is a valid zip file.")
        else:
            print(f"File is not a valid zip file. Header: {header}")
            raise ValueError("Downloaded file is not a valid zip file.")

    # Proceed with extracting if the file is valid
    zip_ref = zipfile.ZipFile(local_zip, 'r')
    zip_ref.extractall('/tmp')
    zip_ref.close()
    print(len(os.listdir('/tmp/PetImages/Cat/')))
    print(len(os.listdir('/tmp/PetImages/Dog/')))
    # Expected Output:
    # 12501
    # 12501

--2025-01-12 08:11:37--  https://download.microsoft.com/download/3/E/1/3E1C3F21-ECDB-4869-8368-6DEBA77B919F/kagglecatsanddogs_3367a.zip
Resolving download.microsoft.com (download.microsoft.com)... 184.29.22.44, 2600:1406:5600:78b::317f, 2600:1406:5600:790::317f
Connecting to download.microsoft.com (download.microsoft.com)|184.29.22.44|:443... connected.
HTTP request sent, awaiting response... 404 Not Found
2025-01-12 08:11:37 ERROR 404: Not Found.

Downloaded file size: 0 bytes
Downloaded file is empty. Please check the download URL and your network connection.
You can find an updated download URL at: https://www.microsoft.com/en-us/download/confirmation.aspx?id=54765
Right-click on the 'Download Manually' link to get a new URL.


In [ ]:
print(len(os.listdir('/tmp/PetImages/Cat/')))
print(len(os.listdir('/tmp/PetImages/Dog/')))

# Expected Output:
# 12501
# 12501

12501
12501


In [9]:
try:
    os.mkdir('/tmp/cats-v-dogs')
    os.mkdir('/tmp/cats-v-dogs/training')
    os.mkdir('/tmp/cats-v-dogs/testing')
    os.mkdir('/tmp/cats-v-dogs/training/cats')
    os.mkdir('/tmp/cats-v-dogs/training/dogs')
    os.mkdir('/tmp/cats-v-dogs/testing/cats')
    os.mkdir('/tmp/cats-v-dogs/testing/dogs')
except OSError:
    pass


In [11]:
import os
import zipfile
import random
import tensorflow as tf
from tensorflow.keras.optimizers import RMSprop
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from shutil import copyfile

# **Verify the Download URL:** Double-check this URL. It might be outdated.
# If the URL doesn't work, visit https://www.microsoft.com/en-us/download/confirmation.aspx?id=54765
# And right click on the 'Download Manually' link to get a new URL to the dataset
# Note: This is a very large dataset and will take time to download
!wget --no-check-certificate "https://download.microsoft.com/download/3/E/1/3E1C3F21-ECDB-4869-8368-6DEBA77B919F/kagglecatsanddogs_3367a.zip" -O "/tmp/cats-and-dogs.zip"
local_zip = '/tmp/cats-and-dogs.zip'

# Check file size to see if it downloaded completely
file_size = os.path.getsize(local_zip)
print(f"Downloaded file size: {file_size} bytes")

# If file size is 0, it means the download failed. Try troubleshooting your network connection
# or find an updated download URL.
if file_size == 0:
    # **Instead of raising an error, print a message and provide instructions.**
    print("Downloaded file is empty. Please check the download URL and your network connection.")
    print("You can find an updated download URL at: https://www.microsoft.com/en-us/download/confirmation.aspx?id=54765")
    print("Right-click on the 'Download Manually' link to get a new URL.")
    # **You can optionally add a line to halt execution here:**
    # import sys; sys.exit(1)
else:
    # Check if the file is actually a zip file
    # by checking the magic number at the beginning of the file
    with open(local_zip, 'rb') as f:
        header = f.read(4)
        if header == b'PK\x03\x04':
            print("File is a valid zip file.")
        else:
            print(f"File is not a valid zip file. Header: {header}")
            raise ValueError("Downloaded file is not a valid zip file.")

    # Proceed with extracting if the file is valid
    zip_ref = zipfile.ZipFile(local_zip, 'r')
    zip_ref.extractall('/tmp')
    zip_ref.close()
    print(len(os.listdir('/tmp/PetImages/Cat/')))
    print(len(os.listdir('/tmp/PetImages/Dog/')))
    # Expected Output:
    # 12501
    # 12501

--2025-01-12 08:14:09--  https://download.microsoft.com/download/3/E/1/3E1C3F21-ECDB-4869-8368-6DEBA77B919F/kagglecatsanddogs_3367a.zip
Resolving download.microsoft.com (download.microsoft.com)... 184.29.22.44, 2600:1406:5600:790::317f, 2600:1406:5600:78b::317f
Connecting to download.microsoft.com (download.microsoft.com)|184.29.22.44|:443... connected.
HTTP request sent, awaiting response... 404 Not Found
2025-01-12 08:14:09 ERROR 404: Not Found.

Downloaded file size: 0 bytes
Downloaded file is empty. Please check the download URL and your network connection.
You can find an updated download URL at: https://www.microsoft.com/en-us/download/confirmation.aspx?id=54765
Right-click on the 'Download Manually' link to get a new URL.


In [12]:
print(len(os.listdir('/tmp/cats-v-dogs/training/cats/')))
print(len(os.listdir('/tmp/cats-v-dogs/training/dogs/')))
print(len(os.listdir('/tmp/cats-v-dogs/testing/cats/')))
print(len(os.listdir('/tmp/cats-v-dogs/testing/dogs/')))
# Expected output:
# 11250
# 11250
# 1250
# 1250

0
0
0
0


In [14]:
model = tf.keras.models.Sequential([
    tf.keras.layers.Conv2D(16, (3, 3), activation='relu', input_shape=(150, 150, 3)),
    tf.keras.layers.MaxPooling2D(2, 2),
    tf.keras.layers.Conv2D(32, (3, 3), activation='relu'),
    tf.keras.layers.MaxPooling2D(2, 2),
    tf.keras.layers.Conv2D(64, (3, 3), activation='relu'),
    tf.keras.layers.MaxPooling2D(2, 2),
    tf.keras.layers.Flatten(),
    tf.keras.layers.Dense(512, activation='relu'),
    tf.keras.layers.Dense(1, activation='sigmoid')
])
model.compile(optimizer=RMSprop(learning_rate=0.001), loss='binary_crossentropy', metrics=['accuracy'])

/usr/local/lib/python3.10/dist-packages/keras/src/layers/convolutional/base_conv.py:107: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


In [15]:
TRAINING_DIR = "/tmp/cats-v-dogs/training/"
train_datagen = ImageDataGenerator(rescale=1.0/255.)
train_generator = train_datagen.flow_from_directory(TRAINING_DIR,
                                                    batch_size=100,
                                                    class_mode='binary',
                                                    target_size=(150, 150))

VALIDATION_DIR = "/tmp/cats-v-dogs/testing/"
validation_datagen = ImageDataGenerator(rescale=1.0/255.)
validation_generator = validation_datagen.flow_from_directory(VALIDATION_DIR,
                                                              batch_size=100,
                                                              class_mode='binary',
                                                              target_size=(150, 150))

# Expected Output:
# Found 22498 images belonging to 2 classes.
# Found 2500 images belonging to 2 classes.

Found 0 images belonging to 2 classes.
Found 0 images belonging to 2 classes.


In [22]:
import os
import zipfile
import random
import tensorflow as tf
from tensorflow.keras.optimizers import RMSprop
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from shutil import copyfile

def split_data(SOURCE, TRAINING, TESTING, SPLIT_SIZE):
    """Splits the data into train and test sets.

    Args:
        SOURCE (str): Path to the source directory containing images.
        TRAINING (str): Path to the training directory.
        TESTING (str): Path to the testing directory.
        SPLIT_SIZE (float): Proportion of data to be used for training.
    """
    files = []
    # Check if the source directory exists before listing its contents
    if os.path.exists(SOURCE):
        for filename in os.listdir(SOURCE):
            file = SOURCE + filename
            if os.path.getsize(file) > 0:
                files.append(filename)
            else:
                print(filename + " is zero length, so ignoring.")
    else:
        print(f"Source directory not found: {SOURCE}")  # Print an informative message
        return  # Exit the function to prevent further errors

    training_length = int(len(files) * SPLIT_SIZE)
    testing_length = int(len(files) - training_length)
    shuffled_set = random.sample(files, len(files))
    training_set = shuffled_set[0:training_length]
    testing_set = shuffled_set[training_length:]

    for filename in training_set:
        this_file = SOURCE + filename
        destination = TRAINING + filename
        copyfile(this_file, destination)

    for filename in testing_set:
        this_file = SOURCE + filename
        destination = TESTING + filename
        copyfile(this_file, destination)


CAT_SOURCE_DIR = "/tmp/PetImages/Cat/"
TRAINING_CATS_DIR = "/tmp/cats-v-dogs/training/cats/"
TESTING_CATS_DIR = "/tmp/cats-v-dogs/testing/cats/"
DOG_SOURCE_DIR = "/tmp/PetImages/Dog/"
TRAINING_DOGS_DIR = "/tmp/cats-v-dogs/training/dogs/"
TESTING_DOGS_DIR = "/tmp/cats-v-dogs/testing/dogs/"

split_size = .9
split_data(CAT_SOURCE_DIR, TRAINING_CATS_DIR, TESTING_CATS_DIR, split_size)
split_data(DOG_SOURCE_DIR, TRAINING_DOGS_DIR, TESTING_DOGS_DIR, split_size)

Source directory not found: /tmp/PetImages/Cat/
Source directory not found: /tmp/PetImages/Dog/


In [26]:
import os
import zipfile
import random
import tensorflow as tf
from tensorflow.keras.optimizers import RMSprop
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from shutil import copyfile

# ... (Your previous code to download and extract the dataset) ...

def split_data(SOURCE, TRAINING, TESTING, SPLIT_SIZE):
    """Splits the data into train and test sets.

    Args:
        SOURCE (str): Path to the source directory containing images.
        TRAINING (str): Path to the training directory.
        TESTING (str): Path to the testing directory.
        SPLIT_SIZE (float): Proportion of data to be used for training.
    """
    files = []
    # Check if the source directory exists before listing its contents
    if os.path.exists(SOURCE):
        for filename in os.listdir(SOURCE):
            file = SOURCE + filename
            if os.path.getsize(file) > 0:
                files.append(filename)
            else:
                print(filename + " is zero length, so ignoring.")
    else:
        print(f"Source directory not found: {SOURCE}")  # Print an informative message
        return  # Exit the function to prevent further errors

    training_length = int(len(files) * SPLIT_SIZE)
    testing_length = int(len(files) - training_length)
    shuffled_set = random.sample(files, len(files))
    training_set = shuffled_set[0:training_length]
    testing_set = shuffled_set[training_length:]

    for filename in training_set:
        this_file = SOURCE + filename
        destination = TRAINING + filename
        copyfile(this_file, destination)

    for filename in testing_set:
        this_file = SOURCE + filename
        destination = TESTING + filename
        copyfile(this_file, destination)


# Updated paths to include 'PetImages'
CAT_SOURCE_DIR = "/tmp/PetImages/Cat/"
TRAINING_CATS_DIR = "/tmp/cats-v-dogs/training/cats/"
TESTING_CATS_DIR = "/tmp/cats-v-dogs/testing/cats/"
DOG_SOURCE_DIR = "/tmp/PetImages/Dog/"
TRAINING_DOGS_DIR = "/tmp/cats-v-dogs/training/dogs/"
TESTING_DOGS_DIR = "/tmp/cats-v-dogs/testing/dogs/"

split_size = .9
split_data(CAT_SOURCE_DIR, TRAINING_CATS_DIR, TESTING_CATS_DIR, split_size)
split_data(DOG_SOURCE_DIR, TRAINING_DOGS_DIR, TESTING_DOGS_DIR, split_size)

# ... (rest of your code) ...

Source directory not found: /tmp/PetImages/Cat/
Source directory not found: /tmp/PetImages/Dog/


In [ ]:
# Here's a codeblock just for fun. You should be able to upload an image here
# and have it classified without crashing
import numpy as np
from google.colab import files
from keras.preprocessing import image

uploaded = files.upload()

for fn in uploaded.keys():

  # predicting images
  path = '/content/' + fn
  img = image.load_img(path, target_size=(150, 150))
  x = image.img_to_array(img)
  x = np.expand_dims(x, axis=0)

  images = np.vstack([x])
  classes = model.predict(images, batch_size=10)
  print(classes[0])
  if classes[0]>0.5:
    print(fn + " is a dog")
  else:
    print(fn + " is a cat")